# Part 1: Pipeline for generating S and Sigma from Pan-UKB LD panel

Currently this pipeline solves for $S$ in each quasi-independent region and saves them in `.h5`, which seems even faster to read than `gds` files. 

Code was tested on Sherlock with
+ `openssl/3.0.7`
+ `julia/1.8.4`
+ `R/4.0.2`
+ `java/11.0.11`
+ `python/3.9.0`

`ml julia/1.8.4 R/4.0.2 openssl/3.0.7 java/11.0.11 python/3.9.0` 

Installing R libraries:
```R
# ml system 
# ml openssl
# ml R/4.0.2
install.packages("BiocManager", repos='http://cran.us.r-project.org')
install.packages("openssl", repos='http://cran.us.r-project.org')
BiocManager::install("gwascat")
BiocManager::install("liftOver")
BiocManager::install("gwascat", lib="~/R")
BiocManager::install("liftOver", lib="~/R")
```

## Quasi-independent regions from ldetect

The original data (genome build hg19) is downloaded from: https://bitbucket.org/nygcresearch/ldetect-data/src/master/

# Script to solve knockoff problem across the genome

+ The code below was used to solve the knockoff problem across different quasi-independent LD blocks
+ It requires 12 different inputs
+ To use this script, copy the file below to a file, e.g. `run_script.jl` and in the terminal, execute
```
julia run_script.jl arg1 arg2 arg3 ...
```

In [ ]:
# ml julia/1.8.4 R/4.0.2 java/11.0.11 python/3.9.0 openssl/3.0.7 system

sleep(60rand()) # prevent too many jobs starting at the same time

using CSV
using DataFrames
using EasyLD
using Knockoffs
using LinearAlgebra
using StatsBase
using DelimitedFiles
using JLD2
using HDF5
using RCall
R"library(liftOver)"

function graphical_group_S(
    bm_file::String, # path to the `.bm` hail block matrix folder
    ht_file::String, # path to the `.ht` hail variant index folder
    chr::String, # 1, 2, ..., 22, or X/Y/M
    start_pos::Int, 
    end_pos::Int, 
    outdir::String;
    m=5,
    tol=0.0001, 
    min_maf=0.01, # only SNPs with maf > min_maf will be included in Sigma
    snps_to_keep=nothing, # list of position (if provided, only SNPs in snps_to_keep will be included in Sigma)
    force_block_diag=true, # whether to reorder row/col of S/Sigma/etc so S is returned as block diagonal 
    add_hg38_coordinates=true, # whether to augment Sigma_info with hg38 coordinates. SNPs that cannot be mapped to hg38 will be deleted
    liftOver_chain_dir::String = "",
    method::Symbol = :maxent,
    group_def::String="hc",
    rk::Number=Inf, # minimum rank of Σ before truncating the remaining eigvals to `min_maf`
    verbose=true
    )
    isdir(outdir) || error("output directory $outdir does not exist!")
    group_def ∈ ["hc", "id"] || error("group_def should be \"hc\" or \"id\"")
    add_hg38_coordinates && !isfile(liftOver_chain_dir) && 
        error("To add hg38 coordinates, a liftOver chain file is required")

    # import Sigma
    import_sigma_time = @elapsed begin
        bm = hail_block_matrix(bm_file, ht_file);
        Sigma, Sigma_info = get_block(bm, chr, start_pos, end_pos, 
            min_maf=min_maf, snps_to_keep=snps_to_keep, enforce_psd=true)
    end

    # append hg38 coordinates to Sigma_info and remove SNPs that can't be converted to hg38
    if add_hg38_coordinates
        Sigma, Sigma_info = augment_hg38(Sigma_info, Sigma, chr, liftOver_chain_dir)
    end

    # define groups and representatives
    def_group_time = @elapsed begin
        groups = group_def == "hc" ? 
            hc_partition_groups(Symmetric(Sigma), cutoff=0.5, linkage=:average) : 
            id_partition_groups(Symmetric(Sigma), rss_target=0.5)
        group_reps = choose_group_reps(Symmetric(Sigma), groups, threshold=0.5)
    end

    # reorder SNPs so D2/S2 is really block diagonal
    if force_block_diag
        rearrange_snps!(groups, group_reps, Sigma, Sigma_info)
    end

    # solve for S
    solve_S_time = @elapsed begin
        # solve S using original Sigma
        S, D, obj = solve_s_graphical_group(Symmetric(Sigma), groups, group_reps, method,
            m=m, tol=tol, verbose=verbose)

        # solve S using modified Sigma (enforcing conditional independence)
        # Sigma2 = cond_indep_corr(Sigma, groups, group_reps)
        # S2, D2, obj2 = solve_s_graphical_group(Symmetric(Sigma2), groups, group_reps, method,
        #     m=m, tol=tol, verbose=verbose)
    end

    # save result in .h5 format
    dir = joinpath(outdir, "chr$chr")
    isdir(dir) || mkpath(dir)
    JLD2.save(
        joinpath(dir, "LD_start$(start_pos)_end$(end_pos).h5"), 
        Dict(
            "S" => S,
            "D" => D,
            "Sigma" => Sigma,
            "SigmaInv" => inv(Sigma),
            "groups" => groups,
            "group_reps" => group_reps,
            "Sigma_reps" => Sigma[group_reps, group_reps],
            "Sigma_reps_inv" => inv(Sigma[group_reps, group_reps])
        )
    )
    CSV.write(joinpath(dir, "Info_start$(start_pos)_end$(end_pos).csv"), Sigma_info)
    open(joinpath(dir, "summary_start$(start_pos)_end$(end_pos).csv"), "w") do io
        println(io, "chr,$chr")
        println(io, "start_pos,$start_pos")
        println(io, "end_pos,$end_pos")
        println(io, "m,$m")
        println(io, "p,", size(Sigma, 1))
        println(io, "nreps,", length(group_reps))
        println(io, "max_group_size,", countmap(groups) |> values |> collect |> maximum)
        println(io, "max_rep_group_size,", countmap(groups[group_reps]) |> values |> collect |> maximum)
        println(io, "import_sigma_time,$import_sigma_time")
        println(io, "def_group_time,$def_group_time")
        println(io, "solve_S_time,$solve_S_time")
    end

    return nothing
end

# use liftOver R package to add hg38 coordinates to Sigma_info
# SNPs that can't be converted, or if chr does not match after conversion, are deleted
function augment_hg38(Sigma_info, Sigma, chr, liftOver_chain_dir)
    pos = Sigma_info[!, "pos"]
    @rput chr pos liftOver_chain_dir
    R"""
    df<-cbind(data.frame(paste0('chr',chr)),pos,pos)
    colnames(df)<-c('chr','start','end')
    temp.Granges<-makeGRangesFromDataFrame(df)
    chain <- import.chain(liftOver_chain_dir)
    converted<-data.frame(liftOver(temp.Granges,chain))
    """
    @rget converted
    success_idx, pos_hg38 = falses(size(Sigma, 1)), Int[]
    for row in eachrow(converted)
        row["seqnames"] == "chr$chr" || continue # check chr match
        success_idx[row["group"]] = true
        push!(pos_hg38, row["start"])
    end
    Sigma_info_new = Sigma_info[success_idx, :]
    Sigma_new = Sigma[success_idx, success_idx]
    Sigma_info_new[!, :pos_hg19] = pos[success_idx]
    Sigma_info_new[!, :pos_hg38] = pos_hg38
    select!(Sigma_info_new, Not(:pos))
    return Sigma_new, Sigma_info_new
end

# function to rearrange the SNP orders to that resulting S2 and D2 actually block diagonal
function rearrange_snps!(groups, group_reps, Sigma, Sigma_info)
    perm = sortperm(groups)
    iperm = invperm(perm)
    groups .= @views groups[perm]
    group_reps .= @views iperm[group_reps]
    sort!(group_reps)
    @assert issorted(groups) && issorted(group_reps)
    Sigma .= @views Sigma[perm, perm]
    Sigma_info .= @views Sigma_info[perm, :]
    return nothing
end

# inputs to this script
bm_file = ARGS[1]                   # e.g. UKBB.EUR.ldadj.bm
ht_file = ARGS[2]                   # e.g. UKBB.EUR.ldadj.variant.ht
population = ARGS[3]
chr = ARGS[4]
start_pos = parse(Int, ARGS[5])
end_pos = parse(Int, ARGS[6])
method = Symbol(ARGS[7]) # maxent, mvr, or sdp
group_def = ARGS[8] # "hc" or "id"
remove_imputed_variants = parse(Bool, ARGS[9])
min_maf = parse(Float64, ARGS[10])
liftOver_chain_dir = ARGS[11]
outdir = ARGS[12]

# import typed SNP positions and ref/alt (used when remove_imputed_variants=true)
bimfile = "/oak/stanford/groups/zihuai/UKB_data/genotyped_call/ukb_cal_chr$(chr)_v2.bim"
typed_df = CSV.read(bimfile, DataFrame, header=false)
snps_to_keep = remove_imputed_variants ? typed_df[!, 4] : nothing

graphical_group_S(bm_file, ht_file, chr, start_pos, 
    end_pos, outdir, snps_to_keep=snps_to_keep, 
    method=method, group_def=group_def, 
    liftOver_chain_dir=liftOver_chain_dir)


## Scripts to submit jobs

+ We need to solve 1703 blocks (for EUR panel), so we automate the job submission process with the script below
+ Each region requires ~5 min to solve
+ The Sherlock cluster accepts maximum of 1000 jobs per user, so we put some `sleep` commands in between job submissions to never exceed this number

Overall, solving all regions should take 2 hours max. 

In [10]:
using CSV, DataFrames

# chr = 1, 2, ..., or 22
# method = maxent, mvr, or sdp
# group_def = "hc" or "id"
function run_repeats(population, chr, method, group_def, remove_imputed_variants,
    min_maf; nsim=Inf)

    # find quasi-independent blocks
    region_file = "/oak/stanford/groups/zihuai/pan_ukb_LD_matrices/LD_block/$(population)_hg19/fourier_ls-all.bed"
    df = CSV.read(region_file, DataFrame)
    idx = findall(x -> x == "chr$chr ", df[!, 1])
    start_pos = df[idx, 2]
    end_pos = df[idx, 3] .- 1

    # prepare input parameters
    script_dir = "/oak/stanford/groups/zihuai/pan_ukb_group_knockoffs"
    liftOver_chain_dir = "/oak/stanford/groups/zihuai/GeneticsResources/LiftOver/hg19ToHg38.over.chain"
    bm_file = "/oak/stanford/groups/zihuai/pan_ukb_LD_matrices/UKBB.$population.ldadj.bm"
    ht_file = "/oak/stanford/groups/zihuai/pan_ukb_LD_matrices/UKBB.$population.ldadj.variant.ht"
    outdir = remove_imputed_variants ? 
        "/oak/stanford/groups/zihuai/pan_ukb_group_knockoffs/$population" : 
        "/oak/stanford/groups/zihuai/pan_ukb_group_knockoffs/$(population)_all"

    # submit 1 job per region
    counter = 0
    for (i, (s, e)) in enumerate(zip(start_pos, end_pos))
        # create .sh file to submit jobs
        filename = "submit.sh"
        open(filename, "w") do io
            println(io, "#!/bin/bash")
            println(io, "#")
            println(io, "#SBATCH --job-name=$(population)_$i")
            println(io, "#")
            println(io, "#SBATCH --time=6:00:00")
            println(io, "#SBATCH --cpus-per-task=1")
            println(io, "#SBATCH --mem-per-cpu=32G")
            println(io, "#SBATCH --partition=owners,normal,candes,zihuai")
            println(io, "#SBATCH --output=$(outdir)/slurms/slurm-%j.out")
            println(io, "")
            println(io, "#save job info on joblog:")
            println(io, "echo \"Job \$JOB_ID started on:   \" `hostname -s`")
            println(io, "echo \"Job \$JOB_ID started on:   \" `date `")
            println(io, "")
            println(io, "# load the job environment:")
            println(io, "ml julia/1.8.4 R/4.0.2 java/11.0.11 python/3.9.0 openssl/3.0.7 system")
            println(io, "export JULIA_DEPOT_PATH=\"/home/groups/sabatti/.julia\"")
            println(io, "")
            println(io, "# run code")
            println(io, "echo 'julia $(script_dir)/solve_blocks.jl $bm_file $ht_file $population $chr $s $e $method $group_def $remove_imputed_variants $min_maf $liftOver_chain_dir $outdir'")
            println(io, "julia $(script_dir)/solve_blocks.jl $bm_file $ht_file $population $chr $s $e $method $group_def $remove_imputed_variants $min_maf $liftOver_chain_dir $outdir")
            println(io, "")
            println(io, "#echo job info on joblog:")
            println(io, "echo \"Job \$JOB_ID ended on:   \" `hostname -s`")
            println(io, "echo \"Job \$JOB_ID ended on:   \" `date `")
            println(io, "#echo \" \"")
        end
        # submit job
        run(`sbatch $filename`)
        println("submitted chr$chr:$s-$e") 
        rm(filename, force=true)
        counter += 1
        counter ≥ nsim && break
    end
end

# test submission of 5 jobs
# population = "EUR"
# chr = 1
# method = "maxent"
# group_def = "hc"
# remove_imputed_variants = true
# min_maf = 0.01
# run_repeats(population, chr, method, group_def, remove_imputed_variants, min_maf, nsim=5)

method = "maxent"
group_def = "hc"
remove_imputed_variants = true
min_maf = 0.01
for population in ["EUR", "AFR", "EAS"]
    for chr in 1:7
        run_repeats(population, chr, method, group_def, remove_imputed_variants, min_maf)
    end
    sleep(600)

    for chr in 8:13
        run_repeats(population, chr, method, group_def, remove_imputed_variants, min_maf)
    end
    sleep(600)

    for chr in 14:22
        run_repeats(population, chr, method, group_def, remove_imputed_variants, min_maf)
    end
    sleep(600)
end

Submitted batch job 26428426
submitted chr1:10583-1976557
Submitted batch job 26428428
submitted chr1:1976558-3492738
Submitted batch job 26428430
submitted chr1:3492739-4543011
Submitted batch job 26428432
submitted chr1:4543012-5914789
Submitted batch job 26428434
submitted chr1:5914790-7365001
Submitted batch job 26428437
submitted chr1:7365002-9002908
Submitted batch job 26428439
submitted chr1:9002909-10755519
Submitted batch job 26428441
submitted chr1:10755520-11708677
Submitted batch job 26428444
submitted chr1:11708678-13810999
Submitted batch job 26428446
submitted chr1:13811000-14893156
Submitted batch job 26428448
submitted chr1:14893157-17875819
Submitted batch job 26428450
submitted chr1:17875820-20304719
Submitted batch job 26428453
submitted chr1:20304720-22145805
Submitted batch job 26428455
submitted chr1:22145806-23916164
Submitted batch job 26428458
submitted chr1:23916165-25516892
Submitted batch job 26428460
submitted chr1:25516893-27400827
Submitted batch job 264

Submit a job to automatically run block above

In [3]:
function submit(command::String, cpus, total_mem; jobname="submit")
    per_cpu_mem = round(Int, total_mem / cpus)
    # create .sh file to submit jobs
    filename = "submit.sh"
    open(filename, "w") do io
        println(io, "#!/bin/bash")
        println(io, "#")
        println(io, "#SBATCH --job-name=$jobname")
        println(io, "#")
        println(io, "#SBATCH --time=24:00:00")
        println(io, "#SBATCH --cpus-per-task=$cpus")
        println(io, "#SBATCH --mem-per-cpu=$(per_cpu_mem)G")
        println(io, "#SBATCH --partition=normal,candes,zihuai")
        println(io, "")
        println(io, "#save job info on joblog:")
        println(io, "echo \"Job \$JOB_ID started on:   \" `hostname -s`")
        println(io, "echo \"Job \$JOB_ID started on:   \" `date `")
        println(io, "")
        println(io, "# load the job environment:")
        println(io, "module load julia/1.8.4")
        println(io, "export JULIA_DEPOT_PATH=\"/home/groups/sabatti/.julia\"")
        println(io, "")
        println(io, "# run code")
        println(io, "echo '$command'")
        println(io, command)
        println(io, "")
        println(io, "#echo job info on joblog:")
        println(io, "echo \"Job \$JOB_ID ended on:   \" `hostname -s`")
        println(io, "echo \"Job \$JOB_ID ended on:   \" `date `")
        println(io, "#echo \" \"")
    end
    # submit job
    run(`sbatch $filename`)
end

command = "julia /oak/stanford/groups/zihuai/pan_ukb_group_knockoffs/submit_solve_blocks.jl"
submit(command, 1, 4, jobname="delegator")

Submitted batch job 26366026


Process(`sbatch submit.sh`, ProcessExited(0))

## Scripts to check for failed jobs and resubmit them

On 4/14/2023, failed 67 / 5109 jobs

In [13]:
using CSV, DataFrames

function submit(command::String, outdir)
    # create .sh file to submit jobs
    filename = "resubmit.sh"
    open(filename, "w") do io
        println(io, "#!/bin/bash")
        println(io, "#")
        println(io, "#SBATCH --job-name=resubmit")
        println(io, "#")
        println(io, "#SBATCH --time=4:00:00")
        println(io, "#SBATCH --cpus-per-task=1")
        println(io, "#SBATCH --mem-per-cpu=64G")
        println(io, "#SBATCH --partition=owners,normal,candes,zihuai")
        println(io, "#SBATCH --output=$(outdir)/slurms/slurm-%j.out")
        println(io, "")
        println(io, "#save job info on joblog:")
        println(io, "echo \"Job \$JOB_ID started on:   \" `hostname -s`")
        println(io, "echo \"Job \$JOB_ID started on:   \" `date `")
        println(io, "")
        println(io, "# load the job environment:")
        println(io, "ml julia/1.8.4 R/4.0.2 java/11.0.11 python/3.9.0 openssl/3.0.7 system")
        println(io, "export JULIA_DEPOT_PATH=\"/home/groups/sabatti/.julia\"")
        println(io, "")
        println(io, "# run code")
        println(io, "echo '$command'")
        println(io, command)
        println(io, "")
        println(io, "#echo job info on joblog:")
        println(io, "echo \"Job \$JOB_ID ended on:   \" `hostname -s`")
        println(io, "echo \"Job \$JOB_ID ended on:   \" `date `")
        println(io, "#echo \" \"")
    end
    # submit job
    run(`sbatch $filename`)
    println("submitted command: $command") 
    rm(filename, force=true)
end

method = "maxent"
group_def = "hc"
remove_imputed_variants = true
min_maf = 0.01
population = "EAS" #["EUR", "AFR", "EAS"]

# find quasi-independent blocks
region_file = "/oak/stanford/groups/zihuai/pan_ukb_LD_matrices/LD_block/$(population)_hg19/fourier_ls-all.bed"
df = CSV.read(region_file, DataFrame)
idx = findall(x -> x == "chr$chr ", df[!, 1])
start_pos = df[idx, 2]
end_pos = df[idx, 3] .- 1

# prepare input parameters
script_dir = "/oak/stanford/groups/zihuai/pan_ukb_group_knockoffs"
liftOver_chain_dir = "/oak/stanford/groups/zihuai/GeneticsResources/LiftOver/hg19ToHg38.over.chain"
bm_file = "/oak/stanford/groups/zihuai/pan_ukb_LD_matrices/UKBB.$population.ldadj.bm"
ht_file = "/oak/stanford/groups/zihuai/pan_ukb_LD_matrices/UKBB.$population.ldadj.variant.ht"
outdir = remove_imputed_variants ? 
    "/oak/stanford/groups/zihuai/pan_ukb_group_knockoffs/$population" : 
    "/oak/stanford/groups/zihuai/pan_ukb_group_knockoffs/$(population)_all"

failed = 0
for chr in 1:22
    idx = findall(x -> x == "chr$chr ", df[!, 1])
    start_pos = df[idx, 2]
    end_pos = df[idx, 3] .- 1
    for (s, e) in zip(start_pos, end_pos)
        LDfile = joinpath(outdir, "chr$chr", "LD_start$(s)_end$(e).h5")
        summary_file = joinpath(outdir, "chr$chr","summary_start$(s)_end$(e).csv")
        info_file = joinpath(outdir, "chr$chr","Info_start$(s)_end$(e).csv")
        if !(isfile(LDfile) && isfile(summary_file) && isfile(info_file))
            command = "julia $(script_dir)/solve_blocks.jl $bm_file $ht_file $population $chr $s $e $method $group_def $remove_imputed_variants $min_maf $liftOver_chain_dir $outdir"
            submit(command, outdir)
            failed += 1
        end
    end
end
println("$population failed $failed jobs")

EAS failed 0 jobs
